In [21]:
from torch_geometric.datasets import TUDataset

dataset = TUDataset(root="./data", name="ENZYMES")
print(dataset[2])

Data(edge_index=[2, 92], x=[25, 3], y=[1])


In [23]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

data = next(iter(train_loader))
print(data)
print(data.batch)

DataBatch(edge_index=[2, 502], x=[124, 3], y=[4], batch=[124], ptr=[5])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3])


In [27]:
import torch.nn as nn 
import torch.nn.functional as F 
from torch_geometric.nn import GCNConv, global_mean_pool

class GNN(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(input_dim, 64)
        self.conv2 = GCNConv(64, 64)
        self.conv3 = GCNConv(64, 64)
        self.fc = nn.Linear(64, num_classes)

    def forward(self, x, edge_index, batch):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = self.conv3(x, edge_index)
        x = global_mean_pool(x, batch)
        x = self.fc(x)
        return x
    
model = GNN(input_dim=3, num_classes=6)




In [28]:
# training the model

import torch.optim as optim

NUM_EPOCHS = 200
LEARNING_RATE = 0.01

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()
model.train()

for epoch in range(NUM_EPOCHS):
    for data in train_loader:
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()


In [32]:
# Predict
import torch

# --- Prediction after training ---
model.eval()  # switch to evaluation mode
all_preds = []

with torch.no_grad():  # no gradients needed
    for data in train_loader:  # or test_loader if you have one
        out = model(data.x, data.edge_index, data.batch)  # forward pass
        preds = out.argmax(dim=1)  # predicted class per graph, in our case 4 prediciton for each batch
        all_preds.append(preds)

all_preds = torch.cat(all_preds, dim=0)  # combine all batch predictions
print("Predictions for all graphs:", all_preds)

Predictions for all graphs: tensor([5, 2, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 2, 5, 3, 2, 5, 5, 5, 5, 5, 2,
        5, 5, 5, 4, 4, 5, 5, 5, 5, 5, 2, 5, 5, 5, 5, 5, 5, 4, 5, 4, 2, 2, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 3, 5, 5,
        5, 4, 5, 2, 5, 5, 5, 5, 5, 2, 5, 2, 2, 5, 5, 2, 5, 3, 5, 2, 5, 5, 5, 2,
        3, 5, 5, 5, 4, 5, 5, 5, 4, 2, 5, 2, 5, 5, 2, 5, 5, 5, 5, 5, 3, 5, 3, 4,
        2, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 2, 5, 3, 5, 5, 5,
        5, 4, 5, 2, 5, 5, 5, 4, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 2,
        5, 5, 5, 5, 5, 5, 5, 5, 2, 5, 5, 5, 4, 4, 5, 4, 5, 5, 5, 5, 5, 5, 5, 3,
        5, 4, 5, 2, 5, 5, 4, 5, 5, 5, 5, 2, 5, 5, 5, 5, 2, 5, 5, 5, 2, 5, 2, 5,
        4, 5, 2, 2, 5, 5, 5, 5, 2, 5, 5, 5, 4, 5, 5, 5, 2, 5, 5, 5, 4, 5, 2, 2,
        5, 5, 4, 5, 2, 5, 5, 5, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 2, 5, 5, 5, 5, 2,
        3, 5, 5, 5, 5, 2, 5, 5, 3, 5, 5, 4, 3, 3, 3, 5, 2, 4, 5, 2, 5, 4, 5, 3,
        5, 2